# Installing libaries

In [1]:
!pip install opencv-python
!pip install sinch
!pip install azure.cognitiveservices.vision.customvision
!pip install --upgrade azure-cognitiveservices-vision-customvision
!pip install azure-identity
!pip install azure-storage-blob azure-identity
!pip install azure-storage-blob[aio]
!pip install azure-storage-blob azure-mgmt-storage
!pip install azure-storage-blob
!pip install numpy

In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid
import cv2
from sinch import Client
from azure.identity.aio import DefaultAzureCredential
from azure.storage.blob.aio import BlobServiceClient, BlobClient, ContainerClient
from azure.storage.blob import PublicAccess
import asyncio
import os
from azure.identity.aio import DefaultAzureCredential
from azure.storage.blob.aio import BlobServiceClient, BlobClient, ContainerClient
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, PublicAccess
import numpy as np
import io
import asyncio
import os
import uuid
from azure.storage.blob import BlobBlock, StandardBlobTier
from azure.identity.aio import DefaultAzureCredential
from azure.storage.blob.aio import BlobServiceClient, ContainerClient, BlobClient
import base64

# CV Model

In [3]:
ENDPOINT = "https://aiepcustom-prediction.cognitiveservices.azure.com/" #model endpoint
prediction_key = "092eec11dfde4b4d8fb81b7505ad36b1" #model key
project_id = "94bd8b74-8c22-4f32-9536-40addd03b945" #project id
publish_iteration_name = "Iteration1" #iteration/model name
iteration_id='c940cb86-4a0a-4fbf-9b23-b36f952f5cb3' #iteration/model id


# trained endpoint that can be used to make a prediction
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

def prediction(image):
    classes=[]
    with open(os.path.join (image), "rb") as image_contents:
        results = predictor.detect_image(
            project_id, publish_iteration_name, image_contents.read())

        for prediction in results.predictions:
            if prediction.probability >= 0.86:  # Set the threshold to 80%
                classes.append(prediction.tag_name)
                print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))
    print("\n")
    unique_classes = set(classes)
    return unique_classes

# SMS API

In [4]:
def sms():
    sinch_client = Client(
        key_id="0e850afa-a134-42c5-ad0d-521e4f05acec",
        key_secret="rritjfPiSK-vaDfJRo4Ek2MwR.",
        project_id="a0b6337f-308c-4752-aa69-8e64e7ce31ee"
    )

    send_batch_response = sinch_client.sms.batches.send(
        body="Elderly detected out of boundary. Click here to see the image: https://playbackaiep.blob.core.windows.net/video/image1.jpg",
        to=["+6598967886"],
        from_="+447520662385",
        delivery_report="none"
    )

    print(send_batch_response)

# Blob Storage

In [5]:

class BlobSamples(object):

    # <Snippet_upload_blob_file>
    async def upload_blob_file(self, blob_service_client: BlobServiceClient, container_name: 'video'):
        container_client = blob_service_client.get_container_client(container=container_name)
        with open(file=os.path.join(directory, filename), mode="rb") as data:
            blob_client = await container_client.upload_blob(name="image1.jpg", data=data, overwrite=True)

In [6]:
# tis code
async def main():
    sample = BlobSamples()

    # TODO: Replace <storage-account-name> with your actual storage account name
    account_url = "https://playbackaiep.blob.core.windows.net"
    credential = 'n5ZXNLlZb21/r15DBNNDgLn2gX9WBK1/6S19oyAlue6OCJV++xZL5flw6aLE7bz0+yFkF8AJKH+4+AStN4kb5g=='

    async with BlobServiceClient(account_url, credential=credential) as blob_service_client:
        await sample.upload_blob_file(blob_service_client, "video")


# Main

In [9]:
vid = cv2.VideoCapture(0)
global base64Frames
base64Frames = []

if not vid.isOpened():
    print("Error: Unable to open webcam.")
    exit()

while True:
    ret, frame = vid.read()

    if not ret:
        print("Error: Unable to capture frame.")
        break

    # Process the frame here (e.g., resize, convert color)
    processed_frame = frame  # Replace this with your processing logic

    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))
    
    cv2.imshow('frame', processed_frame)
    # Save the processed frame as an image file
    if len(base64Frames) % 30 == 0:
        
        directory = "image"

# Specify the filename of your image
        filename = "temp_frame.jpg"
        # Ensure that the directory exists; if not, create it
        if not os.path.exists(directory):
            os.makedirs(directory)

        # Combine the directory and filename to get the full path of the image
        image_path = os.path.join(directory, filename)

        # Assuming 'processed_frame' is your image data, save the image
        cv2.imwrite(image_path, processed_frame)
                
        unique_classes = prediction(image_path)
    
        print("Classes detected:", unique_classes)
        print('\n')
        
        if 'patient' and 'nurse'  in unique_classes:
            print('No SMS')
            
        elif 'patient' in unique_classes:               
            await main()
            sms()
            time.sleep(1)

    if len(base64Frames) % 400 == 0:
        del base64Frames[:100]
    
    elif cv2.waitKey(1) & 0xFF == ord('q'):
        break


vid.release()
cv2.destroyAllWindows()



Classes detected: set()




Classes detected: set()




Classes detected: set()




Classes detected: set()


	patient: 88.73% bbox.left = 0.01, bbox.top = 0.04, bbox.width = 0.48, bbox.height = 0.96


Classes detected: {'patient'}


SendSMSBatchResponse(id='01HQ0GC477BFJ6X66XT1PQ14C5', to=['6598967886'], from_='447520662385', body='Elderly detected out of boundary. Click here to see the image: https://playbackaiep.blob.core.windows.net/video/image1.jpg', delivery_report='none', cancelled=None, type='mt_text', campaign_id=None, created_at='2024-02-19T11:00:16.743Z', modified_at='2024-02-19T11:00:16.743Z', send_at=None, expire_at='2024-02-22T11:00:16.743Z', callback_url=None, client_reference=None, feedback_enabled=None, flash_message=None)


Classes detected: set()




Classes detected: set()




Classes detected: set()




Classes detected: set()


